## Notebook #4 goal

This notebook has the code for variable stay based on the client's situation (aka case). It is working code, but not integrated as of yet into the main model body

For more details on the case, read the main README.md


#### Very basic shelter model

This simpy model is Inspired by car wash example https://simpy.readthedocs.io/en/latest/examples/carwash.html

The parameters in this model are:
- the client is only 1 person 
    - *this need to make it random from 1-6)*
- the stay period of client is random, and between 7 days to 365
    - *this needs to be made into a distribution (or combination of distribution) based on any one of the four cases discussed [here](https://github.com/sim-team-z/learning-simpy/blob/main/README.md)*
- the number of beds in shelter are 35
- the clients arrive at the shelter seeking refuge every 7 days.
    - *this needs to be made variable, perhaps picking an RV from a poisson distribution*


This model also turn away clients when at capacity, and keeps count of the number.

In [9]:
import random
import simpy


RANDOM_SEED = 42
NUM_BEDS = 15  # Number of beds in the shelter
STAYTIME = random.randint(7, 365)      # days the initial clients stayed at shelter
T_INTER = 7       # Create a client every ~7 days
SIM_TIME = 365*3     # Simulation time in days
referred_clients_count = 0 # number of clients turned away. 

class Shelter(object):
    """A shelter has a limited number of beds (``NUM_BEDS``) to
    host clients.

    Clients have to request one of the beds. When they got one, they
    can start their stay (which takes ``staytime`` minutes).

    """
    referred_clients = 0
    
    def __init__(self, env, num_beds, staytime):
        self.env = env
        self.shelter = simpy.Resource(env, num_beds)
        print('Shelter has {} beds available'.format(NUM_BEDS - self.shelter.count))
        self.staytime = staytime
        self.referred_clients = 0

    def get_count_of_referred_clients():
        return self.referred_clients
    
    def increment_count_of_referred_clients():
        self.referred_clients += 1
    
    def stay(self, client):
        """The staying processes. It takes a ``client`` and makes the client stay for a random duration at the shelter."""
        stay_duration = random.randint(7, 365)
        yield self.env.timeout(stay_duration)
        print("Shelter hosted the {} for {} days, now has {} beds free".format(
            client,
            stay_duration, 
            NUM_BEDS - self.shelter.count))
        
    def stay2(self, client, case):
        """Duration of stay for client in the shelter based on her case (CAPABLE=1, CULTURAL=2, MENTAL=3, SKILLS=4)."""
        if case == 1 : # Capable and quickly back on their feet
            stay_duration = random.randint(7, 2*30)
        elif case == 2: #Language and cultural barrier
            stay_duration = random.randint(9*30, 12*30)
        elif case == 3: #Mental barrier
            stay_duration = random.randint(4*30, 9*30) # TODO this is a long tailed distribution, not uniform. check README.md.
            #need to change
        elif case == 4:
            stay_duration = random.randint(2*30, 6*30)
        
        yield self.env.timeout(stay_duration)
        print("Shelter hosted the {}/case type {} for {} days, now has {} beds free".format(
            client,
            case,
            stay_duration, 
            NUM_BEDS - self.shelter.count))

def client(env, name, sh):
    """The client process (each client has a ``name``) arrives at the shelter
    (``sh``) and requests a bed.
    """
    print('%s arrives at the shelter on day %i.' % (name, env.now))
    if (NUM_BEDS - sh.shelter.count == 1):
        print("!!!!SHELTER AT CAPACITY, REFERRING {} TO FAMILY OR HOTEL ROOM".format(name))
        #cw.shelter.increment_count_of_referred_clients()
        sh.referred_clients += 1
        print("Shelter has unfortunately turned {} clients away thus far.".format(sh.referred_clients))
        
    else:      
        with sh.shelter.request() as request:
            yield request

            print('%s enters the shelter on day %i.' % (name, env.now))
            yield env.process(sh.stay(name))

            print('%s leaves the shelter on day %i.' % (name, env.now))


def setup(env, num_beds, staytime, t_inter):
    """Create a clientstay, a number of initial clients and keep creating clients
    approx. every ``t_inter`` minutes."""
    # Create the clientstay
    shelter = Shelter(env, num_beds, staytime)

    # Create 4 initial clients
    for i in range(4):
        env.process(client(env, 'Client %d' % i, shelter))

    # Create more clients while the simulation is running
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(client(env, 'Client %d' % i, shelter))
    print("Number of clients turned away: {}".format(shelter.referred_clients))
    


# Setup and start the simulation
random.seed(RANDOM_SEED)  # This helps reproducing the results

# Create an environment and start the setup process
env = simpy.Environment()
env.process(setup(env, NUM_BEDS, STAYTIME, T_INTER))


# Execute!

env.run(until=SIM_TIME)


Shelter has 15 beds available
Client 0 arrives at the shelter on day 0.
Client 1 arrives at the shelter on day 0.
Client 2 arrives at the shelter on day 0.
Client 3 arrives at the shelter on day 0.
Client 0 enters the shelter on day 0.
Client 1 enters the shelter on day 0.
Client 2 enters the shelter on day 0.
Client 3 enters the shelter on day 0.
Client 4 arrives at the shelter on day 5.
Client 4 enters the shelter on day 5.
Client 5 arrives at the shelter on day 11.
Client 5 enters the shelter on day 11.
Shelter hosted the Client 0 for 19 days, now has 9 beds free
Client 0 leaves the shelter on day 19.
Client 6 arrives at the shelter on day 20.
Client 6 enters the shelter on day 20.
Client 7 arrives at the shelter on day 29.
Client 7 enters the shelter on day 29.
Client 8 arrives at the shelter on day 34.
Client 8 enters the shelter on day 34.
Client 9 arrives at the shelter on day 39.
Client 9 enters the shelter on day 39.
Client 10 arrives at the shelter on day 45.
Client 10 enters